Task 4


In [ ]:
#  dependencies
!pip install  transformers==4.40.2 bitsandbytes==0.43.1 peft==0.11.1 accelerate==0.30.1
!pip install  ranx pandas tqdm sentencepiece
!pip install triton==2.0.0
!pip install torch pytrec_eval pandas numpy
!pip install peft==0.11.1
!pip install bitsandbytes==0.43.1
!pip install transformers==4.40.2
!pip install peft==0.11.1
!pip install accelerate==0.30.1
!pip install torch pytrec_eval pandas numpy tqdm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# libraries

import os, gzip, torch, logging, tqdm, json
import pandas as pd
from collections import defaultdict
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSequenceClassification
import ranx

In [ ]:

logging.basicConfig(level=logging.INFO, format="%(asctime)s  %(message)s")
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# Load queries, qrels, and passage candidates
DATA_DIR = "/content/gdrive/MyDrive/Colab Notebooks"
QUERY_FILE = os.path.join(DATA_DIR, "msmarco-test2019-queries (1).tsv.gz")
TOPK_FILE = os.path.join(DATA_DIR, "msmarco-passagetest2019-top1000.tsv.gz")
QRELS_FILE = os.path.join(DATA_DIR, "2019qrels-pass.txt")


In [ ]:
# Load all queries
all_queries = {}
with gzip.open(QUERY_FILE, "rt") as f:
    for line in f:
        qid, query = line.strip().split("\t")
        all_queries[qid] = query


In [ ]:
#only 43
qrels = ranx.Qrels.from_file(QRELS_FILE, kind="trec")
graded_qids = set(qrels.qrels.keys())
queries = {qid: all_queries[qid] for qid in graded_qids}


In [ ]:
# top-k passages with text
candidates = defaultdict(list)
with gzip.open(TOPK_FILE, "rt") as f:
    for line in f:
        qid, pid, _, passage = line.strip().split("\t", 3)
        if qid in graded_qids:
            candidates[qid].append((pid, passage))

In [ ]:
from transformers import AutoTokenizer, LlamaForCausalLM, AutoModelForCausalLM

model_name = "HuggingFaceH4/zephyr-7b-beta"
tokenizer = AutoTokenizer.from_pretrained(model_name, truncation=True, padding=True, padding_side="left", maximum_length = 2048, model_max_length = 2048)
model = AutoModelForCausalLM.from_pretrained(model_name, load_in_4bit = True, device_map = 'auto')
tokenizer.pad_token = tokenizer.eos_token
model.generation_config.pad_token_id = model.generation_config.eos_token_id

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
# Generate CoT query expansions

def format_cot_prompt(query):
    return (
        "Answer the following query:\n\n"
        f"{query}\n\n"
        "Give the rationale before answering"
    )

cot_reformulations = {}
for qid in tqdm.tqdm(queries):
    prompt = format_cot_prompt(queries[qid])
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.inference_mode():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            temperature=0.7,
            do_sample=False
        )
    full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Truncate to firs 50 words
    reform = " ".join(full_text.split()[:50])
    cot_reformulations[qid] = reform

  0%|          | 0/43 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
  2%|▏         | 1/43 [00:23<16:05, 23.00s/it]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
100%|██████████| 43/43 [10:34<00:00, 14.77s/it]


In [ ]:
#  Load TinyBERT reranker
reranker_name = "cross-encoder/ms-marco-TinyBERT-L-2-v2"
ce_tokenizer    = AutoTokenizer.from_pretrained(reranker_name)
ce_model        = AutoModelForSequenceClassification.from_pretrained(reranker_name).to(device).eval()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/787 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/17.6M [00:00<?, ?B/s]

In [ ]:
def score_passages(query_text, passages):
    pairs = [(query_text, p) for p in passages]
    enc = ce_tokenizer(
        pairs,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    ).to(device)
    with torch.inference_mode():
        logits = ce_model(**enc).logits.squeeze().cpu().tolist()
    return logits



In [ ]:

baseline_run = "/content/baseline_tinybert.run"
cot_run      = "/content/cot_tinybert.run"
with open(baseline_run, "w") as base_f, open(cot_run, "w") as cot_f:
    for qid in tqdm.tqdm(queries):
        pids, texts = zip(*candidates[qid])
        base_scores = score_passages(queries[qid], texts)
        cot_scores  = score_passages(cot_reformulations[qid], texts)
        for rank, (pid, s) in enumerate(zip(pids, base_scores)):
            base_f.write(f"{qid} Q0 {pid} {rank} {s:.4f} BASE\n")
        for rank, (pid, s) in enumerate(zip(pids, cot_scores)):
            cot_f.write(f"{qid} Q0 {pid} {rank} {s:.4f} COT\n")


100%|██████████| 43/43 [00:53<00:00,  1.24s/it]


In [ ]:
# Evaluate baseline vs CoT
metrics = ["ndcg@10", "recall@100", "map@1000"]
run_base = ranx.Run.from_file(baseline_run, kind="trec")
run_cot  = ranx.Run.from_file(cot_run, kind="trec")
fmt = lambda sc: {m: round(v*100, 2) for m, v in sc.items()}
print("\n=== Task 4 Evaluation ===")
print("Baseline:", fmt(ranx.evaluate(qrels, run_base, metrics)))
print("CoT     :", fmt(ranx.evaluate(qrels, run_cot, metrics)))



=== Task 4 Evaluation ===


/usr/local/lib/python3.11/dist-packages/ranx/metrics/ndcg.py:72: NumbaTypeSafetyWarning: unsafe cast from uint64 to int64. Precision may be lost.
  scores[i] = _ndcg(qrels[i], run[i], k, rel_lvl, jarvelin)


Baseline: {'ndcg@10': np.float64(69.84), 'recall@100': np.float64(51.28), 'map@1000': np.float64(46.17)}
CoT     : {'ndcg@10': np.float64(61.46), 'recall@100': np.float64(47.13), 'map@1000': np.float64(43.11)}


In [ ]:
#  Format CoT/PRF prompt
def format_prf_prompt(query, docs):
    return (
        "Answer the following query based on the context:\n\n"
        f"Context: {docs[0]}\n"
        f"         {docs[1]}\n"
        f"         {docs[2]}\n"
        f"Query:   {query}\n\n"
        "Give the rationale before answering"
    )

In [ ]:

docs_to_use = 3
prf_reformulations = {}
for qid in tqdm.tqdm(queries):
    top_docs = [p for _, p in candidates[qid][:docs_to_use]]
    prompt = format_prf_prompt(queries[qid], top_docs)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.inference_mode():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            temperature=0.7,
            do_sample=False
        )
    full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract only the final answer line
    lines = [l.strip() for l in full_text.split("\n") if l.strip()]
    reform_line = lines[-1]
    # Truncate reformulated query to 50 words
    prf_reformulations[qid] = " ".join(reform_line.split()[:50])

100%|██████████| 43/43 [11:34<00:00, 16.14s/it]


In [ ]:

prf_run = "/content/prf_tinybert.run"
with open(prf_run, "w") as prf_f:
    for qid in tqdm.tqdm(queries):
        pids, texts = zip(*candidates[qid])
        prf_scores = score_passages(prf_reformulations[qid], texts)
        for rank, (pid, s) in enumerate(zip(pids, prf_scores)):
            prf_f.write(f"{qid} Q0 {pid} {rank} {s:.4f} PRF\n")

100%|██████████| 43/43 [00:31<00:00,  1.37it/s]


In [ ]:
#  Evaluate baseline vs CoT vs CoT/PRF
run_prf = ranx.Run.from_file(prf_run, kind="trec")
print("\n=== Task 5 Evaluation ===")
runs = {"Baseline": run_base, "CoT": run_cot, "CoT/PRF": run_prf}
for name, run in runs.items():
    print(f"{name:8s}:", fmt(ranx.evaluate(qrels, run, metrics)))



=== Task 5 Evaluation ===
Baseline: {'ndcg@10': np.float64(69.84), 'recall@100': np.float64(51.28), 'map@1000': np.float64(46.17)}
CoT     : {'ndcg@10': np.float64(61.46), 'recall@100': np.float64(47.13), 'map@1000': np.float64(43.11)}
CoT/PRF : {'ndcg@10': np.float64(9.28), 'recall@100': np.float64(16.02), 'map@1000': np.float64(12.29)}
